In [1]:

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

import os
from collections import defaultdict
import time

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
import numpy as np
import pickle
import pandas as pd
import numpy as np
import os
import time
import unidecode
from keras.callbacks import ModelCheckpoint
import random
import editdistance
from itertools import zip_longest

Using TensorFlow backend.


This a very undocumented version of some code I wrote to generate an LSTM auto-encoder for fixing the spelling of Spanish names. It uses names from the INE and a data generator to yield perturbated versions of these  names to a NN (along with the correct version). The LSTM auto-encoder generates an intermediate representation of the  name that´s used  to rebuild it using a decoder.


# Global variables

In [2]:
LSTM_SIZE = 32
SEQUENCE_LENGTH=64

# Load the data and prepare a generator

In [3]:
def pad(string,size =SEQUENCE_LENGTH):
    return "="*(SEQUENCE_LENGTH-len(string)) + string

len(pad("a"))

64

In [10]:
def fullNameGenerator(ratio = 0.2):
    procuradores = []
    
    lastNames = []
    with open("./lastNames.txt",encoding="utf8") as csvfile:
        lastNames = [line.strip() for line in csvfile.readlines()]
    names = []
    with open("./names.txt",encoding="utf8") as csvfile:
        names = [line.strip() for line in csvfile.readlines()]
        
    while True:
        if random.random() < ratio:
            #yield random.sample(procuradores,1)[0]
            yield " ".join(random.sample(names,1)+random.sample(lastNames,2))
        else: 
            yield " ".join(random.sample(names,1)+random.sample(lastNames,2))

In [11]:
nameGenerator = fullNameGenerator()

In [12]:
for i,name in enumerate(nameGenerator):
    print(name)
    if i ==10:
        break
    

CRISTINA GOMEZ SERRANO
ANTONIA PLAZA ESCRIBANO
JUAN FRANCISCO QUESADA APARICIO
VICTOR MANUEL DE LA CRUZ DOMINGUEZ
PEDRO CALERO SIERRA
ROSA MARIA ALARCON BENITEZ
SILVIA CORDERO JUAREZ
ANA BELEN VALLE MOYA
EVA DEL RIO GUZMAN
TOMAS VALVERDE LORENTE
LUIS MIGUEL ANDREU JUAN


# Vectorize the data.

Get the list of input characters

In [13]:
input_characters = set(list("ABCDEFGHIJKLMNOPQRSTUVWXYZ Ñ-Ü_="))
target_characters = set()

with open("./lastNames.txt",encoding="utf8") as csvfile:
    lastNames = [line.strip() for line in csvfile.readlines()]
    for name in lastNames:
        for char in name: 
            input_characters.add(char)

with open("./names.txt",encoding="utf8") as csvfile:
    firstNames = [line.strip() for line in csvfile.readlines()]
    for name in firstNames:
        for char in name: 
            input_characters.add(char)

            
input_characters = sorted(list(input_characters))

numTokens = len(input_characters)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])


# String manipulation functions

In [14]:
def generateError (perturbationFactor, removalFactor = 0.2, insertionFactor = 0.2, modificationFactor = 0.6):
    
    removalFactor*= perturbationFactor
    insertionFactor*= perturbationFactor
    modificationFactor*= perturbationFactor
    
    
    rand = random.random()
    
    if rand < removalFactor:
        return "REMOVAL"
    if rand < insertionFactor+removalFactor:
        if random.random() < 0.5:
            return "INSERTION_AFTER"
        else:
            return "INSERTION_BEFORE"
        
    if rand < insertionFactor+removalFactor+modificationFactor:
        return "MODIFICATION"
    
    return "NONE"

The following function generates random perturbations on a string. This randomness should be changed to adjust it to an OCR/spelling  mistakes distribution

In [16]:
import random
def perturbeSentence(x,factor=0.1, removalFactor = 0.2, insertionFactor = 0.2):
    x = x.upper()
    NOISE = "".join(list(set(input_characters).difference(set("\n")))).upper()
    x = list(x)
    newX = []
    for (pos, noise) in enumerate(random.choice(NOISE) for _ in range(int(len(x)))):
        perturbation = generateError(factor,removalFactor = 0.33, insertionFactor = 0.33, modificationFactor = 0.33)
        
        if perturbation == "NONE":
            newX.append(x[pos])
        
        if perturbation == "REMOVAL":
            pass
        
        if perturbation == "INSERTION_AFTER":
            newX.append(x[pos])
            newX.append(noise)
            
        if perturbation == "INSERTION_BEFORE":
            newX.append(noise)
            newX.append(x[pos])
            
        if perturbation == "MODIFICATION":   
            newX.append(noise)
            
            
    for i in range(len(x)):
        if x[i] not in NOISE+ " ":
            x[i] = ''
            
    x= "".join(newX)
    
    
    return x
perturbeSentence("Francisco Rodríguez Drumond",factor = 0.1)

'FRANCISCO RODQRÍGU=EZDRUMOND'

In [17]:
def generateData(nameGenerator, numTokens, input_token_index,sequenceSize =64, batchSize = 2048,returnStrings=False,factor =0.05):
    while True:
        X = []
        Y = []
        for _ in range(batchSize):
            y = next(nameGenerator)
            
            X.append(pad(perturbeSentence(y,factor =factor)))
            Y.append(pad(y)[::-1])
            
        encoder_input_data = np.zeros(
            (len(X), sequenceSize, numTokens),
            dtype='float32')

        decoder_target_data = np.zeros(
            (len(Y), sequenceSize, numTokens),
            dtype='float32')
    
        for i, (input_text, target_text) in enumerate(zip(X, Y)):
            input_text = input_text[0:64]
            for t, char in enumerate(input_text):
                try:
                    encoder_input_data[i, t, input_token_index[char]] = 1.
                except KeyError:
                    pass # Padding
            for t, char in enumerate(target_text):
                # decoder_target_data is ahead of decoder_input_data by one timestep
                try:
                    decoder_target_data[i, t, input_token_index[char]] = 1.
                except KeyError:
                    pass # Padding
        if returnStrings:
            yield X,Y,encoder_input_data,decoder_target_data
        else:
            yield encoder_input_data,decoder_target_data

Take a look at how it works. The LSTM work if we  reverse  the strings...

In [18]:
gen = generateData(nameGenerator,numTokens,input_token_index,returnStrings=True)
i = 0
for X,Y,_,__ in gen:
    i +=1
    
    for x,y,x_,y_ in zip_longest(X,Y,_,__):
        
        print(y)
        print(x)
        print(x_[40])
        '''
        print(len(x_),len(y_))
        for c1,c2,c3,c4 in zip_longest(x,y,x_,y_):
            print(c1)
            print(c2)
            print(c3)
            print(c4)
        '''

        
        print("")
    if i >=1:
        break

ALLIDAP OREVIR EGROJ============================================
============================================JOQKE RIVERO PADILLA
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

RALIUGA ATAM AIRAM AVE==========================================
=========================================EVA MARJIA MATA AGUILAR
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ARDEVAAS ERRIUGA SACUL==========================================
=========================================LUCAS AGUIFRRE SAAVEDRA
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

AÑEP ASOR DADELOS AIRAM=========================================
=========================================MARIA SOLEGAD ROSA PEÑA
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0

  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ODALLOC EDREVLAV LEAMSI=========================================
=========================================ISMAEL VALVERDE COLLADO
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ELLAV ODNODER AILIME============================================
============================================EMILIA REDONDO VALLE
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

NELLIUG OVARB ATIRAGRAM=========================================
==========================================MARGARITA BRAV GUILLEN
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

AREV OVLAC LEINAD===============================================
===============================================DANIEL CALV CVERA
[ 0.  0.  1.  0.  0.  0. 

=======================================MARSIA RTOFA MARCO RDRIGO
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ADEJO NIRAM ANA=================================================
==================================================ANA MAIN OJEDA
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

NOTNA NOMAR AEREN===============================================
===============================================NEREA RAMON ANTON
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ORAZAL ZEDNANREH AINOTNA========================================
======================================ANTONIA HERNANSDEZW LAZARO
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ZERAUJ AREV CRAM=

=====================================MARIA JOSEFA SIMON BLAZQUEZ
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

OIR LED NOTNA LEUQAR============================================
==========================================RAAQUEL ANTON DEL RIBO
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

LEUGIM ORETERRAC REHTSE=========================================
=========================================ESTHER CARRETERO MIGUEL
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ATSOC ZEDNEM LEAMSI=============================================
=============================================ISMAEL MENDEZ COSTA
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ZEDNEM EUQUL LEGN

SORETSELLAB SOMAR LEUQAR========================================
====================================RAQUEL RAMOSZ BALLECSTZEROJS
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ZEHCNAS OBOC ODERFLA============================================
============================================ALFREDO CO_O SANCHEZ
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ZNAS SOIR DIVAD=================================================
==================================================DAVIDRIOS SANZ
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ATSOC SERDNA OGAITNAS===========================================
===========================================SANTIAGO ANDRES COSTA
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0

...

========================================JAFVIER CARRILLO ESCUDRO
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

LARROC NOMAR SOLRAC NAUJ========================================
=========================================JUAN CARLOS RMON COROAL
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

RODAMA OREDUCSE SEROLOD=========================================
==========================================DOLORES ESCUDEO AMADOR
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

OVLAC SARERTNOC AIRAM AVE=======================================
=======================================EVAW MARIA CONRERAS CALVO
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]

OLOP ZEUGNIMOD ID

ARUGES SAMOT NOICPECNOC AIRAM===================================
====================================MARIA-CONCEPCION TOMASSEGURA
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

SALAS SETROC OINOTNA============================================
===========================================ANTODNIO CORTES SALAS
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

NOCAHC ZURC NELEB ANA===========================================
===========================================ANA BELEN CRUZ CHACON
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ONAC ALLIDAP NAILUJ=============================================
=============================================JULIAN PÜDILLA CANO
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0

ZENITRAM ZERAVLA AIDUALC========================================
=======================================CLAUDIAH ALVAREZ MARTINEZ
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

DEMAHOM RABOCSE AICUL===========================================
==========================================LUCRIA ESCOBAR MOHAMED
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

NADLOR OZNEROL AIRAM ESOJ=======================================
======================================JOSE MARIAP LORENZO ROLDAN
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

SAVIR OTEIRP ODRACIR============================================
===========================================RICARDO PRIETCO RIVAS
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0

  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ALLIDAP ZIAS NAILUJ=============================================
=============================================JULIAN SAIZ PADILLA
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

AGETRO SEYER OIRASOR============================================
==============================================ROSARIO REES RTEG=
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ARIEREP AICARG AICILA===========================================
==========================================ALICIA GRACIA PEXREIRA
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ZEUGNIMOD ETNEROL AIRAM=========================================
=========================================MARIA LZRENTE DOMINGUEZ
[ 0.  0.  1.  0.  0.  0. 


SERDNA ARERREH OIRAM============================================
===========================================MARIO HERORERA ANDRES
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ZERAVLA ODRALLAG SIUL NAUJ======================================
======================================JUAN L_UIS GALLARD ALVAREZ
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

SAM NOGARA LEUGIM===============================================
==============================================MIGUEL ARAGOJN MAS
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

ODASAC ANIDEM LEUGIM ESOJ=======================================
=======================================JOSE MIGUEL MEDINA CASADO
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0.  0.  

# Define NN topology and instantiate model

This code is an adaptation of an autoencoder implementation found on the web.

In [19]:
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, Dropout, recurrent
def generate_model(output_len,numTokens, chars=None,input_layers=2,output_layers=2,hidden_size=512,amount_of_dropout=0.2):
    """Generate the model"""
    print('Build model...')
    model = Sequential()
    
    for layer_number in range(input_layers):
        model.add(recurrent.LSTM(hidden_size, input_shape=(None, numTokens), kernel_initializer="he_normal",
                                 return_sequences=layer_number + 1 < input_layers))
        model.add(Dropout(amount_of_dropout))
    
    model.add(RepeatVector(output_len))
    
    for _ in range(output_layers):
        model.add(recurrent.LSTM(hidden_size, return_sequences=True, kernel_initializer="he_normal"))
        model.add(Dropout(amount_of_dropout))

    
    model.add(TimeDistributed(Dense(numTokens, kernel_initializer="he_normal")))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [20]:
encoder_input =  Input(shape=(SEQUENCE_LENGTH,numTokens))
encoder_lstm = Bidirectional(LSTM(LSTM_SIZE,return_sequences=True))(encoder_input)

l_lstm_sent = Bidirectional(LSTM(LSTM_SIZE,return_sequences=True))(encoder_lstm)
preds = TimeDistributed(Dense(numTokens, activation='softmax'))(l_lstm_sent)
model = Model(encoder_input, preds)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
model = generate_model(SEQUENCE_LENGTH,numTokens)

Build model...


In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, None, 512)         1116160   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 64, 512)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64, 512)           2099200   
_________________________________________________________________
dropout_3 (Dropout)          (None, 64, 512)           0         
__________

In [23]:
from keras.callbacks import EarlyStopping
filepath="./autoencoder-0.5/completed_05_perturbation_weights-improvement-b-{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint,EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')]


# Prepare generators and fit model.

In [24]:
train_gen = generateData(nameGenerator,numTokens,input_token_index,returnStrings=False,batchSize=160,sequenceSize =SEQUENCE_LENGTH)
cv_gen =    generateData(nameGenerator,numTokens,input_token_index,returnStrings=False,batchSize=160,sequenceSize =SEQUENCE_LENGTH)

In [ ]:
model.fit_generator(train_gen,
                    steps_per_epoch = 100,
                    epochs=5000000,
                    validation_data=cv_gen,
                    callbacks=callbacks_list,
                    #nb_val_samples=1000,
                    validation_steps= 100)

# Test

### Load best model.

In [ ]:
model.load_weights("./autoencoder-0.5/completed_05_perturbation_weights-improvement-b-308-0.02-1.00.hdf5")

In [26]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
#reverse_input_char_index[0] = ""
reverse_input_char_index

{0: ' ',
 1: '-',
 2: '=',
 3: 'A',
 4: 'B',
 5: 'C',
 6: 'D',
 7: 'E',
 8: 'F',
 9: 'G',
 10: 'H',
 11: 'I',
 12: 'J',
 13: 'K',
 14: 'L',
 15: 'M',
 16: 'N',
 17: 'O',
 18: 'P',
 19: 'Q',
 20: 'R',
 21: 'S',
 22: 'T',
 23: 'U',
 24: 'V',
 25: 'W',
 26: 'X',
 27: 'Y',
 28: 'Z',
 29: '_',
 30: 'Ñ',
 31: 'Ü'}

In [27]:
def encode(strings,sequenceSize = SEQUENCE_LENGTH):

    encoder_input_data = np.zeros(
        (len(strings), sequenceSize, numTokens),
        dtype='float32')
    
    for i, input_text in enumerate(strings):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.
    #print(np.shape(encoder_input_data))
    return encoder_input_data

def decode(encodeds):
    decoded = []
    
    for i, encoded in enumerate(encodeds):
        reconstructedString = ""
        for characterCandidateVector in encoded:
            sampled_token_index = np.argmax(characterCandidateVector)
            reconstructedString += reverse_input_char_index[sampled_token_index]
            
        decoded.append(reconstructedString)
        
    return decoded

In [28]:
testGen = generateData(nameGenerator,numTokens,input_token_index,returnStrings=True,batchSize=160,sequenceSize =SEQUENCE_LENGTH,factor=0.2)

In [ ]:
i = 0

changedPerturbationCorrect = 0
kept = 0
changedNoPerturbation = 0
changedPerturbationWrong = 0

perturbed = 0 
nonPerturbed = 0 

truePred = []
trueChanged  = []

for X,Y,encoder_input_data,decoder_target_data in testGen:
    i +=1
    
    for x,y,xV,_ in zip(X,Y,encoder_input_data,decoder_target_data):

        #print("Correct: " + y[::-1])
        #print("Changed: " + x)
        #print(model.predict(np.asarray([xV])))
        '''
        reconstructedString = ""
        for characterCandidateVector in model.predict(np.asarray([xV]))[0]:
            sampled_token_index = np.argmax(characterCandidateVector)
            reconstructedString += reverse_input_char_index[sampled_token_index]
        '''
        
        reconstructedString = decode([model.predict(np.asarray([xV]))[0]])[0]
        
        #print("Output:  " +str(reconstructedString))
        #print("Output:  " +reconstructedString[::-1])
        
        
        truePred.append(editdistance.eval(y, reconstructedString))
        trueChanged.append(editdistance.eval(y[::-1], x))
        
        for trueChar, changedChar,predChar in zip_longest(y,x,reconstructedString):
            
            
            if trueChar != changedChar:
                perturbed+=1
            else:
                nonPerturbed+= 1
                
            if trueChar == predChar:
                if trueChar != changedChar:
                    changedPerturbationCorrect+=1
                else:
                    kept +=1
            else:
                if trueChar == changedChar:
                    
                    changedNoPerturbation +=1
                else:
                    changedPerturbationWrong+=1
        #print("===")        
    if i >=10:
        break
print(mistakes)

In [ ]:
print("Levenschtein distance True-Perturbated " +str(np.mean(trueChanged)))
print("Levenschtein distance True-Predicted   " + str(np.mean(truePred)))